In [ ]:
"""
The purpose of this script is to automate SLA tasks. This includes creating folders for each month and establishment,
creating emails and letters from templates, and more. 


Process:
1) Copy list of items from online agenda into text doc as shown below
2) I will likely want to quickly remove certain text from this, such as section descriptions like 'New Liquor License Applications,' or other info that will not go into the final outputs like the description of the license type. 
3) The most important thing will likely be keeping each entry to one line in the input file as I will likely use line breaks to parse each item. 
4) If this doesn't work, another option might be the period (.) followed by blank spaces. 
5) DBA format seems to be in parentheses after the name. 
6) I will have to manually add lawyers names.
7) Agenda number is the number in the list itself. 

----------------------------------------------------------------------------
1 - Open and read lines of text file
    - https://www.geeksforgeeks.org/reading-writing-text-files-python/
2 - Iterate through each line and build a dataframe / table from the relevant contents
3 - Filter out rows that begin with numeric. These should be the agenda items. 
4 - split at period and get first item, which is the numeric agenda item number. 
5 - split again and get the business name
6 - split again and get the address

---------------------------------------------------------------------------------

TO DO
 - Create code to remove illegal characters from strings (for example 'B'way')
 - Start scripting work to input values into template letters
 - Create output for excel, which can serve a number of functions:
     - a place to input additional info like lawyers name
     - track other SLA items
 - Create similar script that takes as an input an excel file instead of a text file. 


"""

In [1]:
import pandas as pd
import datetime as dt
from datetime import datetime
import openpyxl
import os
import csv

In [ ]:
agenda_pull = r"C:\Users\MN03\Desktop\Calvin Docs\SLA\Automation Work\Example_Agenda_Pull.txt"

In [2]:
def make_sla_folders(agenda_pull):
    
    # Open the text file
    #r+ = read/write access mode
    agenda = open(agenda_pull, 'r+')
    
    #Readlines creates a list, where each index contains a line of text, which in this case is a single establishment. 
    contents = agenda.readlines()
    
    # Close the text file
    agenda.close()
    
    # This creates a dataframe where each row is a line from the agenda pull
    agenda_df = pd.DataFrame(contents, columns=['line'])
    
    # This line creates new column that contains a bool series with 'True' for every line that starts with a a digit
    # Lines that do not contain a digit (agenda number) will be removed.
    agenda_df['entry_row']= agenda_df['line'].str[0].str.isdigit()
    
    # Filters out rows that do not contain an agenda item (start with a digit). 
    # Because the entry_row column is boolean, just calling it as as a filter will remove False entries.  
    agenda_df = agenda_df[agenda_df.entry_row]
    
    # Create new column with agenda number only for each row
    agenda_df['agenda_number'] = agenda_df.loc[:,'line'].str.split(pat=".").str[0]
    
    # Create column with all agenda item info except for the agenda item number
    agenda_df['agenda_info_no_number'] = agenda_df.loc[:,'line'].str.split(pat=".").str[1]
    
    # This partitions the string into:
    # 1: business name
    # 2: comma
    # 3: address, and license type and notes
    # Once partitioned the 1st and 3rd columns are output. 
    agenda_df['b_name'] = agenda_df.loc[:,'agenda_info_no_number'].str.partition(sep=",", expand=True)[0]
    agenda_df['address'] = agenda_df.loc[:,'agenda_info_no_number'].str.partition(sep=",", expand=True)[2]
    
    # This creates a column showing the text in the first parentheses. The second set is not important because these will always be
    # notes on the liquor licence, which aren't important for this exercise. 
    agenda_df['address_sup'] = agenda_df['address'].str.extract('\(([^)]+)')

    # The first set of parentheses contains either an address supplement, such as 'basement',
    # or information about the liquor license. 
    # Strings containing 'op' or 'wb' are filtered out in the code below so that only address supplementary info remains. 
    agenda_df['address_sup3'] = agenda_df['address_sup'].str.contains('op|wb', na=True)
    
    # Replace all values in the address supplement column identified above as having 'op' or 'wb' with an empty string.
    agenda_df['address_sup'] = agenda_df['address_sup'].mask(agenda_df['address_sup3'], "")
    
    # This line splits out the address and creates a new string with everything to the left of the first 
    # opening parenthesis, which is the primary address. 
    agenda_df['prim_address'] = agenda_df['address'].str.split(pat="(").str[0]
    
    # This line removes '\n' characters from each row
    #df = df.replace('\n','', regex=True)
    agenda_df['prim_address'] = agenda_df['prim_address'].replace('\n','', regex=True)
    
    # This line splits out the business name and creates a new string with everything to the left of the first 
    # opening parenthesis, which is the business trade name.
    agenda_df['b_tradename'] = agenda_df['b_name'].str.split(pat="(").str[0]
    
    # This creates a column showing the text in the first parentheses. The second set is not important because these will always be
    # notes on the liquor licence, which aren't important for this exercise. 
    agenda_df['b_llc_name'] = agenda_df['b_name'].str.extract('\(([^)]+)')

    # This replaces NAN with empty string
    agenda_df['b_llc_name'] =agenda_df['b_llc_name'].fillna('')
    
    # Current month (number and name) and year. This will be used to create top level folder. 
    month_name = str(datetime.now().strftime("%B"))
    month_num = str(datetime.now().month)
    year = str(datetime.now().year)
    
    # This line creates the top level directory with the month, year, and 'SLA'
    month_dir = month_num + '-' + month_name + ' ' + year + ' SLA'

    
    #Print message that script is running
    print("Making SLA folders at following location:")
    
    # FIND DESKTOP PATH and create a folder structure below it. 
    desktop = os.path.expanduser("~/Desktop")
    top_folder = 'SLA_AUTO_OUTPUT'
    filepath = os.path.join(desktop, top_folder, month_dir)
    os.makedirs(filepath)
    print(filepath) 
    
    # Strip whitespace from left and right of column. Consider doing this for other columns     
    agenda_df['b_tradename'] = agenda_df.b_tradename.str.strip() 
    agenda_df['b_llc_name'] = agenda_df['b_llc_name'].str.strip()
    agenda_df['prim_address'] = agenda_df['prim_address'].str.strip()
    
    
    # 1 Make new folder path for each establishment
    # 2 This will be the primary address followed by a dash followed by the trade name if it exists, else the LLC name
    #      These will follow this pattern: '45 Avenue B - Lamias Fish Market'
    #
    for index, row in agenda_df.iterrows():
        est_filepath = ''
        if row.b_tradename != '':
            est_filepath = row.prim_address + ' - ' + row.b_tradename
        else:
            est_filepath = row.prim_address + ' - ' + row.b_llc_name

        fin_filepath = os.path.join(desktop, top_folder, month_dir, est_filepath)
        os.makedirs(fin_filepath)
        
    return agenda_df;
    

In [ ]:
september_pull =  r"C:\Users\MN03\Desktop\Calvin Docs\SLA\Automation Work\September_Agenda_Pull.txt"

In [ ]:
make_sla_folders(agenda_pull)

In [ ]:
agenda_df = make_sla_folders(agenda_pull)

In [10]:
mac_agenda = r'/Users/calvindechicago/Desktop/SLA_test.txt'

In [11]:
make_sla_folders(mac_agenda)

Making SLA folders at following location:
/Users/calvindechicago/Desktop/SLA_AUTO_OUTPUT/8-August 2021 SLA


,line,entry_row,agenda_number,agenda_info_no_number,b_name,address,address_sup,address_sup3,prim_address,b_tradename,b_llc_name
0,"2. Tre (Jersey Boys LLC), 173 Ludlow St (op...",True,2,"Tre (Jersey Boys LLC), 173 Ludlow St (op/m...",Tre (Jersey Boys LLC),173 Ludlow St (op/method of operation change:...,,True,173 Ludlow St,Tre,Jersey Boys LLC
1,3. Hwa Yuan Szechuan (42-44 East Broadway R...,True,3,Hwa Yuan Szechuan (42-44 East Broadway Res...,Hwa Yuan Szechuan (42-44 East Broadway Res...,42-44 E B'way (op/alt: add 3rd and 4th floors...,,True,42-44 E B'way,Hwa Yuan Szechuan,42-44 East Broadway Restaurant Inc
2,"4. Roberta's (Avenue A Pizza LLC), 15 Ave A...",True,4,"Roberta's (Avenue A Pizza LLC), 15 Ave A (...",Roberta's (Avenue A Pizza LLC),15 Ave A (op)\n,,True,15 Ave A,Roberta's,Avenue A Pizza LLC
3,"5. Bar and Events 14th Street LLC, 124 E 14...",True,5,"Bar and Events 14th Street LLC, 124 E 14th...",Bar and Events 14th Street LLC,124 E 14th St (op)\n,,True,124 E 14th St,Bar and Events 14th Street LLC,
4,"6. 10 Delancey Cocina Inc, 10 Delancey St (...",True,6,"10 Delancey Cocina Inc, 10 Delancey St (op)\n",10 Delancey Cocina Inc,10 Delancey St (op)\n,,True,10 Delancey St,10 Delancey Cocina Inc,
5,"7. French 37 LLC, 37 Canal St (op)\n",True,7,"French 37 LLC, 37 Canal St (op)\n",French 37 LLC,37 Canal St (op)\n,,True,37 Canal St,French 37 LLC,
6,"8. Dali NYC LLC, 37 Market St (op)\n",True,8,"Dali NYC LLC, 37 Market St (op)\n",Dali NYC LLC,37 Market St (op)\n,,True,37 Market St,Dali NYC LLC,
7,"9. Spain East Inc, 49 Ave B (op)\n",True,9,"Spain East Inc, 49 Ave B (op)\n",Spain East Inc,49 Ave B (op)\n,,True,49 Ave B,Spain East Inc,
8,"10. Gama Lounge LLC, 50 Ave B (op)\n",True,10,"Gama Lounge LLC, 50 Ave B (op)\n",Gama Lounge LLC,50 Ave B (op)\n,,True,50 Ave B,Gama Lounge LLC,
9,11. David's (David Hospitality Group Corpor...,True,11,David's (David Hospitality Group Corporati...,David's (David Hospitality Group Corporation),76 Forsyth St (op)\n,,True,76 Forsyth St,David's,David Hospitality Group Corporation


In [9]:
"""
MAKE SLA TRACKING SHEET
- Every month a new tracking sheet needs to be created. 
- Use code above to pull information from the agenda and append it to columns in an excel template.
- The following columns are created:
    Agenda Number
    Business name
    DBA
    Address
    Address Supplement
    Rep Name 
    Rep Email
    Rep Phone
    Sent Date
    Received 
"""

'\nMAKE SLA TRACKING SHEET\n- Every month a new tracking sheet needs to be created. \n- Use code above to pull information from the agenda and append it to columns in an excel template.\n- The following columns are created:\n    Agenda Number\n    Business name\n    DBA\n    Address\n    Address Supplement\n    Rep Name \n    Rep Email\n    Rep Phone\n    Sent Date\n    Received \n'

In [28]:
agenda_df

['INFO PULLED OFF ONLINE AGENDA:\n',
 "2.    Hwa Yuan Szechuan, 42-44 E B'way (op/alt: add additional floors/method of operation: add karaoke)\n",
 '3.    Bridgeview Hotel, 50 Bowery \n',
 'New Liquor License Applications\n',
 '4.    Forsythia (JDS Restaurant LLC), 9 Stanton St (upgrade to op)\n',
 "5.    Roberta's (Avenue A Pizza LLC), 15 Ave A (op)\n",
 '6.    Lucky Lindon, 21 Essex St (op)\n',
 '7.    Avenue C Hospitality LLC, 102 Ave C (op)\n',
 '8.    Bar and Events 14th Street LLC, 124 E 14th St (op)\n',
 '9.    Lucky Star (Parkside 3 NYC LLC), 135 Division St (wb)\n',
 '10.    Moneygoround Inc, 235 Eldridge St (op)\n',
 '11.    Sally Can Wait LLC, 252 Broome St (op)\n',
 '12.    8282 (BBRH LLC), 84 Stanton St (op)\n',
 '13.    Sour Mouse (Sweatshop LES LLC), 110 Delancey St (Basement) (upgrade to op)\n',
 '14.    Tokyo Suhi 168 Inc, 151 Rivington St (op)\n',
 '15.    Henry Street Pretzels LLC, 193 Henry St (1st Floor) (op)\n',
 '16.    ACES (Fine Food and Spirits Inc), 197 2nd A